# Import Packages

In [ ]:
!pip install pydicom

In [ ]:
import pydicom
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import random
import os

import torch
import torch.nn as nn

from torchvision.transforms import v2

from sklearn.model_selection import train_test_split

# DICOM file

1.   檢視: [ImageJ](https://imagej.nih.gov/ij/download.html)
2.   讀取: pydicom


In [ ]:
# A configurable is a regular Python class that serves as a base class for all main classes in an application.
class config: 
    root = "/kaggle/input/2024dl"
    batch_size = 16
    seed = 42

## DICOM information
1. File Meta Information：例如 SOP Class UID、Instance UID、Transfer Syntax UID 等，描述該影像檔的格式和識別資訊。
2. Patient Information：包含病人的名字、年齡、性別、出生日期等資料。
3. Image Information：包括影像的類型（如核醫學影像）、時間（檢查日期、取得影像的時間）、檢查描述（如腦部掃描）、影像幀數（20 幀）、每像素的樣本數（1，即單色影像），影像尺寸（128x128），Pixel Spacing 等。
4. Radiopharmaceutical Information：這部分顯示了使用的藥物劑量與相關的放射性核素資料。
5. Pixel Data：最終列出了影像的像素數據，包括每幀影像的最低與最高像素值等資訊。

In [ ]:
dicom_path = '/DICOM/A1406/00010019'
dicom_file = pydicom.dcmread(config.root + dicom_path)
print(dicom_file)

In [ ]:
# Window Width：視窗的寬度，用來控制影像的對比度。   
# Window Center：視窗的中心位置，用來控制影像的亮度。

print(f"({dicom_file.WindowCenter}, {dicom_file.WindowWidth})")

In [ ]:
images = dicom_file.pixel_array
print(images.shape)

In [ ]:
# 每位 patient 的 slice 數不一定一樣，這邊畫出這位 patient 的 20 張 slice

plt.figure(figsize = (16, 20))
for i in range(images.shape[0]):
    plt.subplot(5,4,i+1)
    
    # 若 imshow 加入 cmap = 'gray'，則可以得到原先灰階的 slice 圖。
    plt.imshow(images[i])
    
    plt.title(f'slice {i+1}')
    plt.axis('off')
    plt.subplots_adjust(wspace = None, hspace = None)

# train.csv


In [ ]:
train_data = pd.read_csv(os.path.join(config.root, 'train.csv'))
train_data

# test.csv

In [ ]:
test_data = pd.read_csv(os.path.join(config.root, 'test.csv'))
test_data

# Pytorch Dataset


1. getitem: 讓定義的類別可以迭代調用。
2. call: 讓定義的類別可以像普通函數一樣，直接用類別名稱(參數)就可以呼叫。

In [ ]:
class ParkinsonsDataset(torch.utils.data.Dataset):

    def __init__(self, df, transforms = None): # 將所有資料提出
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, x):
        
        age = torch.tensor(self.df.iloc[x, 1]) # 年齡
        gender = torch.tensor(self.df.iloc[x, 2]) # 性別
        index = torch.tensor(self.df.iloc[x, 4]) # 起始張數
        label = torch.tensor(self.df.iloc[x, 5]) # 標籤
        
        # 影像前處理
        image = pydicom.dcmread(config.root + self.df.iloc[x, 3]).pixel_array
        
        # DICOM 文件中的圖像數據通常以整數 (int16, uint16 等) 格式存儲，為了確保後續計算的精度及穩定性，所以使用浮點數運算。
        image = torch.tensor(image.astype(np.float32))
        image = image[index-1:index+2, :, :] # 取指定張數和前後共 3 張，image size = [3, 128, 128]
#         image = image[[index]*3, :, :] # 取指定張數重疊共 3 張
        
        if self.transforms: image = self.transforms(image)
        
        return image, age, gender, label

## Transformation

In [ ]:
# v2 中內建的影像處理 function

dir(v2)

1. Min-Max Normalization 適合於將圖像值縮放到固定範圍時，特別是在進行像素範圍變換或需要穩定化不同圖像來源的數據時。
2. v2 的 normalize 更適合於深度學習中的圖像預處理，特別是在訓練神經網絡時，它可以使模型更好地學習特徵而不受像素值範圍的影響。

In [ ]:
# 因為這邊是將 3 張灰階圖去做疊加所得到的 3 維圖，因此若是用 v2 內建的 normalize 可能會不合，但仍可嘗試看看。

class Min_Max_Normalization(object): # 自定義的類別
  # def __init__(self): # 宣告此類別中會使用到的資料

  def __call__(self, image): # 定義有image之後要進行的資料處理
    changed_image = (image - image.min())/(image.max() - image.min())
    return changed_image

Transforms = v2.Compose([
    
    # 在v2中裡的 function
    # 這邊可能要注意有可能切到想要的圖像
    v2.CenterCrop(size = (50, 50)),
    
    # 自定義的 function
    Min_Max_Normalization()
])

# EDA (Exploratory Data Analysis)

In [ ]:
dataset = ParkinsonsDataset(train_data, transforms = Transforms)
test_dataset = ParkinsonsDataset(test_data, transforms = Transforms)

In [ ]:
print(f"Number of training samples: {len(dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

In [ ]:
train_labels = train_data.iloc[:, 5].value_counts().sort_index()
train_labels

## Bar chart

In [ ]:
from matplotlib.ticker import MaxNLocator

labels, counts = train_labels.index, train_labels.values

plt.figure(figsize = (6, 6))

# 設定 zorder 為 2，讓長條圖在網格線上面顯示
plt.bar(labels, counts, color = 'skyblue', zorder = 2)

plt.xlabel('Labels')
plt.ylabel("Number of samples")
plt.title("Number of Samples Per Labels in training dataset")

# 設定 x 軸為整數
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.grid(True, which = 'both', axis = 'both', linestyle = '--', color='gray', linewidth = 0.7, zorder = 1)


# 顯示圖表
plt.show()

In [ ]:
sub_df = train_data[(train_data['Gender'] == 1) & (train_data['Stage'] == 1)]
print('男性且判斷為第1期的人數: ', sub_df.shape[0])

In [ ]:
male = [train_data[(train_data['Gender'] == 1) & (train_data['Stage'] == i)].shape[0] for i in range(1, 4)]
female = [train_data[(train_data['Gender'] == 0) & (train_data['Stage'] == i)].shape[0] for i in range(1, 4)]
print('male: ', male, ', female: ',female)
x_axis = np.arange(len(labels))
plt.bar(x_axis - 0.2, male, 0.4, label = 'Male', color = 'cornflowerblue')
plt.bar(x_axis + 0.2, female, 0.4, label = 'Female', color = 'lightcoral')
plt.xticks(x_axis, labels)
plt.xlabel("Stages")
plt.ylabel("Number of samples")
plt.title("Number of samples in each group")
plt.legend()
plt.show()

## Box plot

orange line：代表該組的中位數，顯示年齡的中間值。  
box：顯示數據的第1四分位數（Q1）和第3四分位數（Q3）之間的範圍，為數據中間的50%。  
Whiskers：表示數據的範圍，不包括離群值。上鬚延伸至Q3+1.5IQR（四分位距），下鬚延伸至Q1-1.5IQR。  
Outliers (dots)：圖中圓點是超出 Whiskers 範圍的數據點，代表可能的離群值或極端值。  

In [ ]:
sub_df = [train_data[train_data['Stage'] == i]['Age'] for i in range(1, 4)]
plt.boxplot(sub_df)
plt.xlabel("Stages")
plt.ylabel("Number of samples")
plt.title("Number of samples in each group")
plt.show()

In [ ]:
def outliers_IQR(data):
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data < lower_bound) | (data > upper_bound)]

# 每組數據的極端值
for i, age_data in enumerate(sub_df, start=1):
    outliers = outliers_IQR(age_data)
    print(f"Stage {i} 的離群值: {outliers.values}")

In [ ]:
train_dataset, val_dataset = train_test_split(train_data, train_size = 0.8, test_size = 0.2, random_state = config.seed)

train_dataset = ParkinsonsDataset(train_dataset, transforms = Transforms)
val_dataset = ParkinsonsDataset(val_dataset, transforms = Transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = config.batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = config.batch_size, shuffle = False)

因為 batch_size 設為 16 ，因此 dataloader 每次迭代中都是用 16 個樣本(patient)，而每位 patient 的 image 都是由 3 張 slice 組成，所以這邊畫出48張圖。

In [ ]:
images, ages, genders, labels = next(iter(train_loader))

plt.figure(figsize = (12, 64)) # (寬, 長)

for i, (image ,label) in enumerate(zip(images, labels)):
    
    for j in range(3):
        
        plt.subplot(16, 3, i*3 +j+1)
        plt.title(f"label {label.numpy()}")
        plt.imshow(image[j])
        plt.axis('off')
        plt.subplots_adjust(wspace = None, hspace = None)

print(f"Image shape: {images.shape}, label shape {labels.shape}")